# Libraries

In [1]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
import pyodbc
import pandas.io.formats.excel
from multiprocessing import Pool
from USA_DWH import main

# Settings
pd.io.formats.excel.ExcelFormatter.header_style = None
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)

# Paramaters

weeks = []
a = datetime(2021, 1, 3)
for i in range(0, 3):
    b = a + timedelta(days=84)
    weeks.append((a.strftime('%Y%m%d'), b.strftime('%Y%m%d')))
    a = b + timedelta(days=7)

weeks = []
a = datetime(2017, 1, 1)
for i in range(0, 124):
    b = a + timedelta(days=7)
    weeks.append((a.strftime('%Y%m%d'), b.strftime('%Y%m%d')))
    a = b + timedelta(days=7)

In [2]:
country = 'Estados Unidos de América'

#weeks = [('20210103', '20210228'), ('20210307', '20210425'), ('20210502', '20210627'), ('20210704', '20210829'), ('20210905', '20210926')]

## SQL Server

In [3]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=no;')

In [4]:
# Connection to DWH
## Server:SFEDWH01|DB:Gnm_DWH
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

## Server:SFEDWH01|DB:Gnm_MasterOp
conn2 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_MasterOp;'
                     'Trusted_Connection=yes;')

# Import data from DWH

In [5]:
# Query time data
qtiempo = '''SELECT TmpID, TmpFecha, TmpSemanaAnioGenomma, TmpMesGenomma, TmpAnioSemanaGenomma
FROM Gnm_DWH.dbo.Dim_Tiempo WHERE TmpAnioSemanaGenomma IN (2018, 2019, 2020, 2021, 2022)'''
# Import time data
dtiempo = pd.read_sql(qtiempo, conn1)

# Query clients data
qclientes= '''SELECT DISTINCT SUC.SucID, SUC.SucCodCliente, CLIE.GrpID, CLIE.GrpNombre, CLIE.CadID, CLIE.CadNombre, SUC.SucNombre
 FROM (SELECT DISTINCT PaisNombre, GrpID, GrpNombre, CadID, CadNombre FROM Gnm_MasterOp.dbo.vw_EstructuraClientesSegPTVTotal
	WHERE PaisNombre = '{0}') AS CLIE
 LEFT JOIN (SELECT DISTINCT SucID, SucNombre, SucCodCliente,CadId FROM Gnm_MasterOp.dbo.vw_EstructuraSucursalesTotal ) AS SUC
	ON CLIE.CadID = SUC.CadID
	ORDER BY SucId;'''

# Query skus data
qproductos = '''SELECT DISTINCT MrcNombre, ProID, ProNombre, ProPstCodBarras, ProPstNombre, ProPstID
                FROM Gnm_MasterOp.dbo.vw_EstructuraProductosInternacional
                WHERE TipoComNombre = '{0}' '''

c:\Users\jshernandezm\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


qsemfalta = 'SELECT TmpID FROM Gnm_DWH.dbo.Dim_Tiempo  WHERE TmpID BETWEEN 20211003 AND 20211205 AND TmpDiaSemana = 7'
#qsemfalta = 'SELECT TmpID FROM Gnm_DWH.dbo.Dim_Tiempo  WHERE TmpID = 20211128 AND TmpDiaSemana = 7'
semfalta = pd.read_sql(qsemfalta, conn1)

In [6]:
%%time
# Import clients data
dclientes = pd.read_sql(qclientes.format(country), conn2)
# Clean data
dclientes.dropna(axis=0, inplace=True)
dclientes.reset_index(drop=True, inplace=True)

# Import products
dproductos = pd.read_sql(qproductos.format(country), conn2)

c:\Users\jshernandezm\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\jshernandezm\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Wall time: 14.3 s


weeks = list(zip(semfalta['TmpID'].tolist(), semfalta['TmpID'].tolist()))

# Merges

In [7]:
params = []
weeks = [(20220220, 20220227), (20220306, 20220313)]
for winit, wend in weeks:
    first_day = winit
    last_day = wend
    params.append([first_day, last_day, dtiempo, dclientes, dproductos])

In [8]:
%%time
pool = Pool(len(params))
final = pool.starmap(main, params)
pool.close()
pool.join()
del pool

Wall time: 1min 37s


In [9]:
data_final = pd.DataFrame()
for table in final:
    data_final = pd.concat([data_final, table])

In [10]:
data_final['Semana GL'] = data_final['Semana GL'].map(int)

In [11]:
data_final.pivot_table(index='GrpNombre', 
                       columns='Semana GL', 
                       values=['Unidades Ventas'],
                       margins=True,
                       aggfunc='sum')

Unidades Ventas                                 
Semana GL                         8       9      10      11      All
GrpNombre                                                           
AMAZON US                     14385   14800   15857   14920    59962
MIDWAY IMPORTING             100514   71268   87725   91946   351453
VERMONT IP BEVERAGE           37986   45395   43074   33224   159679
Wal-Mart USA                  65745   69694   72272   73126   280837
Walgreens USA                 57684   55657   58385   52908   224634
All                          276314  256814  277313  266124  1076565

In [12]:
data_final.pivot_table(index='GrpNombre', 
                       columns='Semana GL', 
                       values=['Monto Ventas'],
                       margins=True,
                       aggfunc='sum')

Monto Ventas                                         \
Semana GL                      8            9           10           11   
GrpNombre                                                                 
AMAZON US             100,920.96   106,524.32   116,385.57   115,568.66   
MIDWAY IMPORTING      387,641.13   282,073.73   343,998.08   346,191.91   
VERMONT IP BEVERAGE    53,493.95    63,815.70    60,565.40    46,786.80   
Wal-Mart USA          373,484.28   402,247.99   423,590.37   427,661.09   
Walgreens USA         361,228.89   351,196.23   370,857.58   334,304.16   
All                 1,276,769.21 1,205,857.97 1,315,397.00 1,270,512.62   

                                  
Semana GL                    All  
GrpNombre                         
AMAZON US             439,399.51  
MIDWAY IMPORTING    1,359,904.85  
VERMONT IP BEVERAGE   224,661.85  
Wal-Mart USA        1,626,983.73  
Walgreens USA       1,417,586.86  
All                 5,068,536.80

In [13]:
data_final[data_final['GrpNombre']!='MIDWAY IMPORTING'].pivot_table(index='GrpNombre', 
                                                                    columns='Semana GL', 
                                                                    values=['Monto Ventas'],
                                                                    margins=True,
                                                                    aggfunc='sum')

Monto Ventas                                              
Semana GL                      8          9         10         11          All
GrpNombre                                                                     
AMAZON US             100,920.96 106,524.32 116,385.57 115,568.66   439,399.51
VERMONT IP BEVERAGE    53,493.95  63,815.70  60,565.40  46,786.80   224,661.85
Wal-Mart USA          373,484.28 402,247.99 423,590.37 427,661.09 1,626,983.73
Walgreens USA         361,228.89 351,196.23 370,857.58 334,304.16 1,417,586.86
All                   889,128.08 923,784.24 971,398.92 924,320.71 3,708,631.95

# Export

%%time
path = '../../../02Paises/EUA/Consultas/'
filename = 'Consulta {0}_{1}-{2}.xlsx'.format(country, datetime.today().strftime('%Y%m%d'), datetime.today().strftime('%H%M'))
data_final.to_excel(path + filename, index=False, sheet_name=country)

# Sankey

In [14]:
from pdb import set_trace
import plotly.graph_objects as go
import pandas as pd
import random
pd.options.display.float_format = '{:,.2f}'.format

In [15]:
def create_params(df, cols, value):
    labels = df.melt(value_vars=cols).drop_duplicates()['value'].tolist()
    labels_nodes = dict(zip(labels, range(len(labels))))
    source, target, values = [], [], []
    link_color = dict(zip(range(len(labels)), [random_color('rgba') for x in range(len(labels))]))
    for sour, targ in zip(cols, cols[1:]):
        aux = df.pivot_table(index=[sour, targ], values=value, aggfunc='sum').reset_index()
        source += aux[sour].map(labels_nodes).tolist()
        target += aux[targ].map(labels_nodes).tolist()
        values += aux[value].tolist()
    df_trace = pd.DataFrame([source, target, values]).T.rename({0:'source', 1:'target', 2:'value'}, axis=1)
    #df_trace['color'] = df_trace['source'].apply(lambda x: random_color('color'))
    df_trace['link color'] = df_trace['source'].map(link_color)
    return df_trace, list(labels_nodes.keys())

In [16]:
def random_color(method):
    if method == 'rgba':
        return 'rgba' + str(tuple(random.choices(range(256), k=3) + [round(random.uniform(0.2, 0.85), 4)]))
    else:
        return "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])

In [17]:
def sankey_diagram(data_trace, labels, year, weeks):
    fig = go.Figure(data=[go.Sankey(orientation = "h",
                                    valueformat = ",.2f",
                                    node = dict(pad = 15, 
                                                thickness = 20, 
                                                line = dict(color = "black", width = 0.5),
                                                label = labels,
                                                #color = data_trace['color']),
                                                color = 'black'),
                                    link = dict(source = data_trace['source'],
                                                target = data_trace['target'],
                                                value = data_trace['value'],
                                                color = data_trace['link color'])
                                   )],
                    layout={'showlegend': True})
    weeks_text = ', '.join([str(w) for w in weeks])
    fig.update_layout(title_text="Genomma USA - Sell Out Week {1} - {0}".format(year, weeks_text), font_size=12)
    fig.show()

In [18]:
def create_params_perc(df, cols, value):
    labels = df.melt(value_vars=cols).drop_duplicates()['value'].tolist()
    labels_nodes = dict(zip(labels, range(len(labels))))
    source, target, values = [], [], []
    link_color = dict(zip(range(len(labels)), [random_color('rgba') for x in range(len(labels))]))
    for sour, targ in zip(cols, cols[1:]):
        aux = df.pivot_table(index=[sour, targ], values=value, aggfunc='sum').reset_index()
        source += aux[sour].map(labels_nodes).tolist()
        target += aux[targ].map(labels_nodes).tolist()
        total_values = aux[value].sum()
        values += [x/total_values for x in aux[value].tolist()]
    df_trace = pd.DataFrame([source, target, values]).T.rename({0:'source', 1:'target', 2:'value'}, axis=1)
    df_trace['link color'] = df_trace['source'].map(link_color)
    return df_trace, list(labels_nodes.keys())

In [19]:
data_final.columns

Index(['Anio GL', 'Semana GL', 'Mes GL', 'Fecha', 'GrpID', 'GrpNombre',
       'CadID', 'CadNombre', 'MrcNombre', 'ProID', 'ProNombre', 'ProPstID',
       'ProPstCodBarras', 'ProPstNombre', 'Unidades Ventas',
       'Unidades Inventario', 'Monto Ventas', 'Monto Inventario'],
      dtype='object')

In [20]:
data_final['País'] = 'USA'

In [21]:
year = 2022
#weeks = range(36, 39)
weeks = [10]
cols_nodes = ['País', 'MrcNombre']

#data = data_final[(data_final['Año_GL'] == year)&(data_final['Semana_GL'].isin(weeks))&(data_final['MrcNombre']=='TIO NACHO')].copy()
data = data_final[(data_final['Anio GL'] == year)&(data_final['Semana GL'].isin(weeks))].copy()
data.reset_index(drop=True, inplace=True)

trace, label = create_params_perc(df=data, cols=cols_nodes, value='Monto Ventas')
sankey_diagram(trace, label, year, weeks)

In [22]:
year = 2022
#weeks = range(36, 39)
weeks = [5]
cols_nodes = ['País', 'MrcNombre']

#data = data_final[(data_final['Año_GL'] == year)&(data_final['Semana_GL'].isin(weeks))&(data_final['MrcNombre']=='TIO NACHO')].copy()
data = data_final[(data_final['Anio GL'] == year)&(data_final['Semana GL'].isin(weeks))&(data_final['GrpNombre'].isin(['Target', 'Walgreens USA', 'Wal-Mart USA', 'AMAZON US', 'VERMONT IP BEVERAGE']))].copy()
data.reset_index(drop=True, inplace=True)

trace, label = create_params_perc(df=data, cols=cols_nodes, value='Monto Ventas')
sankey_diagram(trace, label, year, weeks)

KeyError: 'Monto Ventas'